## Этап 1. Установка библиотек и импорт данных

In [ ]:
# Cкачиваем необходимые библиотеки для работы
!pip install pandas numpy scikit-learn catboost xgboost optuna


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedGroupKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, StackingClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
import pandas as pd
import glob
import os
import numpy as np
import optuna
from optuna.samplers import TPESampler
import time
from sklearn.feature_selection import SelectFromModel

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Подключаемся в Google Drive, чтобы работать с файлами из Google Colaboratory

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Путь к папке с тренировочными данными
path_train = '/content/drive/MyDrive/Кейс-1. Кредитный скоринг юридических лиц/train'
# Путь к папке с тестовыми данными
path_test = '/content/drive/MyDrive/Кейс-1. Кредитный скоринг юридических лиц/test'

Получим список файлов в каждой из папок с данными

In [ ]:
print(*[f'Список файлов в репозитории {x.split("/")[-1]}: {sorted(os.listdir(x), key=lambda x: len(x))}' for x in [path_train, path_test]], sep='\n')

Список файлов в репозитории train: ['train_4.csv', 'train_9.csv', 'train_3.csv', 'train_2.csv', 'train_8.csv', 'train_5.csv', 'train_7.csv', 'train_6.csv', 'train_1.csv', 'train_10.csv']
Список файлов в репозитории test: ['test_5.csv', 'test_3.csv', 'test_4.csv', 'test_2.csv', 'test_9.csv', 'test_7.csv', 'test_6.csv', 'test_8.csv', 'test_1.csv', 'test_10.csv']


Объединим тренировочные и тестовые данные в единые датасеты train и test

In [ ]:
# Получим список путей к файлам в папке train
filenames_train = glob.glob(path_train + "/*.csv")

# Создадим список для записи считанных файлов train
data_files_train = []

# Считаем все файлы train и добавим их в список
for filename in filenames_train:
    data_files_train.append(pd.read_csv(filename))

# Объединим тренировочные данные в единый датасет
data_train = pd.concat(data_files_train, ignore_index=True)

# Выведем информацию о размерности полученных тренировочных данных
display('Размерность полных тренировочных данных составляет: {} строка и {} столбец'.format(*data_train.shape))

# Выведем первые 5 строк тренировочных данных
display(data_train.head())

# Удостоверимся, что перед нами данные только из выборки train
data_train['smpl'].value_counts(dropna=False)

'Размерность полных тренировочных данных составляет: 38731 строка и 421 столбец'

,target,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_409,feature_410,feature_411,feature_412,feature_413,feature_414,feature_415,feature_416,feature_417,feature_418
0,0,train,11619,-0.803605,1.373680,1.0,-0.147178,-1.135569,-0.829743,-0.688879,...,2.002449,1.139800,1.0,0.299604,0.739050,-0.870988,-1.643177,0.259418,3.224089,-0.905515
1,0,train,11620,-2.066605,-1.666932,0.0,0.087665,0.316779,0.918004,-0.143202,...,-2.153467,-0.122552,0.0,0.978408,-0.335446,0.698805,1.920243,0.397668,-0.979179,-1.496148
2,0,train,11621,-1.419976,0.570124,0.0,3.117520,-1.767073,1.087783,1.596354,...,0.541124,-2.008305,0.0,0.437926,-0.052971,1.839667,1.872591,0.065527,1.620158,0.235264
3,0,train,11622,0.241241,-1.165185,0.0,-0.946474,0.288386,0.908839,1.049859,...,-0.298835,0.239494,0.0,-0.797018,-0.289733,0.836043,0.675707,0.293216,0.566113,0.311053
4,0,train,11623,-0.062594,0.446797,0.0,-0.039862,0.305150,-0.087690,-0.157288,...,-0.910352,-0.607310,0.0,1.142165,-1.211827,-0.241350,-0.354882,-1.209285,0.545290,0.092345


,count
smpl,
train,38731


In [ ]:
# Получим список путей к файлам в папке test
filenames_test = glob.glob(path_test + "/*.csv")

# Создадим список для записи считанных файлов test
data_files_test = []

# Считаем все файлы test и добавим их в список
for filename in filenames_test:
    data_files_test.append(pd.read_csv(filename))

# Объединим тестовые данные в единый датасет
data_test = pd.concat(data_files_test, ignore_index=True)

# Выведем информацию о размерности полученных тестовых данных
display('Размерность полных тренировочных данных составляет: {} строк и {} столбцов'.format(*data_test.shape))

# Выведем первые 5 строк тренировочных данных
display(data_test.head())

# Удостоверимся, что перед нами данные только из выборки test
data_test['smpl'].value_counts(dropna=False)

'Размерность полных тренировочных данных составляет: 19826 строк и 420 столбцов'

,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_409,feature_410,feature_411,feature_412,feature_413,feature_414,feature_415,feature_416,feature_417,feature_418
0,test,46659,-1.164883,-0.009703,0.0,1.615347,-1.361970,1.376917,1.225641,0.885135,...,-0.350823,0.316319,0.0,-0.842084,-0.917498,1.575508,2.674942,2.326245,0.713261,-0.794598
1,test,46660,0.298416,0.080642,0.0,-1.565635,-1.297520,-1.249686,0.876467,-1.536158,...,-1.344292,-0.536342,0.0,0.918482,-0.261112,-2.196441,-0.203098,0.817365,-0.955153,0.972887
2,test,46661,-1.233124,0.882072,0.0,0.200780,0.547180,0.729748,0.999526,1.017688,...,0.186111,1.246292,0.0,-1.214631,0.665874,0.701446,0.120424,-2.753440,0.288816,1.168083
3,test,46662,-0.372428,0.175114,0.0,-1.303881,0.555596,0.763836,1.118690,0.140343,...,1.624891,0.682413,0.0,-0.124788,-0.465817,1.138807,0.124537,0.085071,1.201724,-1.038282
4,test,46663,-1.620763,-0.000855,0.0,0.060219,1.137647,0.178389,1.910471,0.203672,...,1.125216,1.367920,0.0,-0.544753,1.413505,0.207799,1.558147,0.167964,-1.254438,-1.415781


,count
smpl,
test,19826


## Этап 2. Работа с моделью

In [ ]:
# Выделим признаки для обучения и искомый признак в датасете train
# для обучения модели
X = data_train.drop(['target', 'smpl'], axis=1)
y = data_train['target']


clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)

model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_new.shape







# Разобьем тренировочные данные на валидационный и тренировочный датасет в соотношении 7 к 3
# Cтратифицируем выборки по искомому признаку, зафиксируем random_state для воспроизводимости
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=0.3)
SEED = 42
gpu = False
cv = StratifiedGroupKFold(5, shuffle=True)



# def cb_objective(trial):
#     params = {
#         'loss_function':     'Logloss',
#         'iterations':        trial.suggest_int('iterations', 100, 3000),
#         'verbose':           False,
#         'random_state':      SEED,
#         'learning_rate':     trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
#         'max_depth':         trial.suggest_int('max_depth', 2, 8),
#         'l2_leaf_reg':       trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
#         'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 1.0),
#         'min_data_in_leaf':  trial.suggest_int('min_data_in_leaf', 5, 100),
#         # 'scale_pos_weight':  trial.suggest_float('scale_pos_weight', 1.0, 50.0),
#         'auto_class_weights': 'Balanced',
#     }

#     if gpu:
#       params['task_type'] = 'GPU'
#       params['colsample_bylevel'] = None


#     estimator = CatBoostClassifier(**params)

#     val_score = cross_val_score(
#         estimator=estimator,
#         X=X_train, y=y_train,
#         cv=cv,
#         scoring='roc_auc',
#     )

#     return np.mean(val_score)

# def xgb_objective(trial):
#     params = {
#         'objective':          'binary:logistic',
#         'n_estimators':       trial.suggest_int('iterations', 100, 3000),
#         'tree_method':        'hist',
#         'random_state':       SEED,
#         'learning_rate':      trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
#         'max_depth':          trial.suggest_int('max_depth', 2, 8),
#         'lambda':             trial.suggest_float('lambda', 1e-3, 10.0, log=True),
#         'alpha':              trial.suggest_float('alpha', 1e-3, 10.0, log=True),
#         'subsample':          trial.suggest_float('subsample', 0.4, 1.0),
#         'colsample_bytree':   trial.suggest_float('colsample_bytree', 0.4, 1.0),
#         'colsample_bynode':   trial.suggest_float('colsample_bynode', 0.4, 1.0),
#         # 'scale_pos_weight':   trial.suggest_float('scale_pos_weight', 1.0, 50.0),
#     }

#     if gpu:
#       params['device'] = 'cuda'

#     estimator =  XGBClassifier(**params)

#     val_score = cross_val_score(
#         estimator=estimator,
#         X=X_train, y=y_train,
#         cv=cv,
#         scoring='roc_auc',
#     )

#     return np.mean(val_score)

# def lgb_objective(trial):
#     params = {
#         'objective':         'binary',
#         'verbosity':         -1,
#         'n_iter': trial.suggest_int('iterations', 100, 3000),
#         'boosting_type':  'gbdt',
#         'lambda_l1':         trial.suggest_float('lambda_l1', 1e-3, 10.0, log=True),
#         'lambda_l2':         trial.suggest_float('lambda_l2', 1e-3, 10.0, log=True),
#         'learning_rate':     trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
#         'max_depth':         trial.suggest_int('max_depth', 2, 8),
#         'num_leaves':        trial.suggest_int('num_leaves', 16, 256),
#         'colsample_bytree':  trial.suggest_float('colsample_bytree', 0.4, 1.0),
#         'colsample_bynode':  trial.suggest_float('colsample_bynode', 0.4, 1.0),
#         'bagging_fraction':  trial.suggest_float('bagging_fraction', 0.4, 1.0),
#         'bagging_freq':      trial.suggest_int('bagging_freq', 1, 7),
#         'min_data_in_leaf':  trial.suggest_int('min_data_in_leaf', 5, 100),
#         # 'scale_pos_weight' : trial.suggest_float('scale_pos_weight', 1.0, 50.0),
#         'is_unbalance': True,
#     }

#     # if gpu:
#     #   params['device'] = 'gpu'

#     estimator =  LGBMClassifier(**params)


#     val_score = cross_val_score(
#         estimator=estimator,
#         X=X_train, y=y_train,
#         cv=cv,
#         scoring='roc_auc',
#     )

#     return np.mean(val_score)


# DO_TUNING = True

# if DO_TUNING:
#     # start_time = time.time()
#     # study_cb = optuna.create_study(direction='maximize', sampler=TPESampler(seed=SEED))
#     # study_cb.optimize(cb_objective, n_trials=100, )
#     # end_time = time.time()
#     # elapsed_time_cb = end_time - start_time
#     # print(f"CatBoost tuning took {elapsed_time_cb:.2f} seconds.")

#     start_time = time.time()
#     study_lgb = optuna.create_study(direction='maximize', sampler=TPESampler(seed=SEED))
#     study_lgb.optimize(lgb_objective, n_trials=100)
#     end_time = time.time()
#     elapsed_time_lgb = end_time - start_time
#     print(f"LightGBM tuning took {elapsed_time_lgb:.2f} seconds.")

#     start_time = time.time()
#     study_xgb = optuna.create_study(direction='maximize', sampler=TPESampler(seed=SEED))
#     study_xgb.optimize(xgb_objective, n_trials=100)
#     end_time = time.time()
#     elapsed_time_xgb = end_time - start_time
#     print(f"XGBoost tuning took {elapsed_time_xgb:.2f} seconds.")

#     print("Best LGBM trial:", study_lgb.best_trial, 'Best LGBM params', study_lgb.best_params)
#     # print("Best CatBoost trial:", study_cb.best_trial, "Best CatBoost params:", study_cb.best_params)
#     print("Best XGBoost trial:", study_xgb.best_trial, "Best XGBoost params:", study_xgb.best_params)







/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [ ]:
lgbm_best_params = {'iterations': 364,
                    'lambda_l1': 0.010704832348942385,
                    'lambda_l2': 0.0010550603312438993,
                    'learning_rate': 0.01844097043806042,
                    'max_depth': 4,
                    'num_leaves': 221,
                    'colsample_bytree': 0.8656114831191837,
                    'colsample_bynode': 0.7216731059131875,
                    'bagging_fraction': 0.6314632033193033,
                    'bagging_freq': 7,
                    'min_data_in_leaf': 71,
                    'is_unbalance': True,
                    'boosting_type':  'gbdt',
                    'objective': 'binary',
                    }


xgb_best_params = {'iterations': 814,
                    'learning_rate': 0.04396292285758932,
                    'max_depth': 2,
                    'lambda': 5.299997901477308,
                    'alpha': 0.004501306460529177,
                    'subsample': 0.9788710816897642,
                    'colsample_bytree': 0.955629520902894,
                    'colsample_bynode': 0.7382293731620369,
                    'objective': 'binary:logistic',
                   }


cb_best_params = { 'loss_function': 'Logloss',
                   'verbose': False,
                   'random_state': SEED,
                   'auto_class_weights': 'Balanced'
                  }




# estimators = []
# # estimators.append(('LGBM', LGBMClassifier(**lgbm_best_params)))
# estimators.append(('CatBoost', CatBoostClassifier(**cb_best_params)))
# estimators.append(('XGB', XGBClassifier(**xgb_best_params)))
# # # final = LogisticRegression()

# gbdt_clf = StackingClassifier(estimators=estimators,cv=6)
# # gbdt_clf = Ca(tBoostClassifierloss_function='Logloss', random_state=SEED, auto_class_weights='Balanced')
# gbdt_clf.fit(X_train, y_train)
# # # # Обучим модель на тренировочной части тренировочных данных
# # # gbdt_clf = RandomForestClassifier(n_estimators=100,
# # #                                oob_score=True,
# # #                                random_state=1).fit(X_train, y_train)

# # # Получим предсказание с вероятностями для валидационной части тренировочного датасета
# # catboost_model = CatBoostClassifier(n_estimators=4000,
# #                                     random_seed=42,
# #                                     max_depth=8,
# #                                     early_stopping_rounds=40,
# #                                     eval_metric='AUC',
# #                                     loss_function='Logloss').fit(X_train, y_train)




cat_model = CatBoostClassifier(**cb_best_params)
xgb_model = XGBClassifier(**xgb_best_params)
lgbm_model = LGBMClassifier(**lgbm_best_params)

stacking_1_base_models = [
    ('logreg', LogisticRegression()),
    ('knn3', KNeighborsClassifier(n_neighbors=3)),
    ('knn5', KNeighborsClassifier(n_neighbors=5)),
    ('knn7', KNeighborsClassifier(n_neighbors=7)),
    ('knn9', KNeighborsClassifier(n_neighbors=9)),
    ('knn12', KNeighborsClassifier(n_neighbors=12)),
    ('knn15', KNeighborsClassifier(n_neighbors=15)),
    ('tree3', DecisionTreeClassifier(max_depth=3)),
    ('tree5', DecisionTreeClassifier(max_depth=5)),
    ('tree7', DecisionTreeClassifier(max_depth=7)),

]

stacking_2_base_models = [
    #('rf', RandomForestClassifier()),
    ('gb', GradientBoostingClassifier()),
    ('ada', AdaBoostClassifier()),
    # ('et', ExtraTreesClassifier())
]

stacking_3_base_models = [
    ('lda', LinearDiscriminantAnalysis()),
    ('qda', QuadraticDiscriminantAnalysis())
]

stacking_4_base_models = [
    ('xgb', xgb_model),
    # ('lgbm', lgbm_model),
    ('catboost', cat_model)
]


# Создание стэкингов
stacking_1 = StackingClassifier(
    estimators=stacking_1_base_models,
    final_estimator=MLPClassifier(hidden_layer_sizes=(256, 128, 64), alpha=0.001, random_state=SEED, n_iter_no_change=30),
    cv=2
)

stacking_2 = StackingClassifier(
    estimators=stacking_2_base_models,
    final_estimator=LogisticRegression(),
    cv=2
)

stacking_3 = StackingClassifier(
    estimators=stacking_3_base_models,
    final_estimator=LogisticRegression(),
    cv=2
)

stacking_4 = StackingClassifier(
    estimators=stacking_4_base_models,
    final_estimator=LogisticRegression(),
    cv=2
)

# Объединение стэкингов в финальную модель
# Для этого нам нужно сначала обучить каждый стэкинг по отдельности
stacking_1.fit(X_train, y_train)
# stacking_2.fit(X_train, y_train)
# stacking_3.fit(X_train, y_train)
stacking_4.fit(X_train, y_train)

# Получение мета-признаков
meta_features_1 = stacking_1.predict(X)
# meta_features_2 = stacking_2.predict(X)
# meta_features_3 = stacking_3.predict(X)
meta_features_4 = stacking_4.predict(X)

# Создание финальной модели
final_meta_features = np.column_stack((meta_features_1, meta_features_4))
final_model = MLPClassifier()

stacking_final = StackingClassifier(
    estimators=[
        ('st1', stacking_1),
        #('st2', stacking_2),
        #('st3', stacking_3),
        ('st4', stacking_4),
        # ('svm', SVC()),
        ('xgb', xgb_model),
        # ('lgbm', lgbm_model)
    ],
    final_estimator=MLPClassifier(hidden_layer_sizes=(256, 128, 64), alpha=0.001, random_state=SEED, n_iter_no_change=30),
    cv=2
).fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [ ]:
y_pred = stacking_final.predict_proba(X_test)
y_pred = pd.Series(y_pred[:,1])

# Высчитаем метрику roc-auc по валидационным данным
print(f'Метрика roc-auc на валидационных данных имеет значение: {roc_auc_score(y_test, y_pred)}')
# print(f'Метрика accuracy: {accuracy_score(y_test, y_pred)}')

Метрика roc-auc на валидационных данных имеет значение: 0.7857027363378701


## Этап 3. Предсказание для тестовых данных



Получим предсказание на тестовых данных

In [ ]:
# # Обучим модель на полных тренировочных данных
# gbdt_clf = RandomForestClassifier(n_estimators=100,
#                                oob_score=True,
#                                random_state=1).fit(X, y)

# Для предсказания используем тестовый датасет с исключенным признаком smpl
y_test_pred = stacking_final.predict_proba(data_test.drop(['smpl'], axis=1))

# Переведем предсказание в формат Series
y_test_pred = pd.Series(y_test_pred[:, 1])

# Добавим данные предсказания к датасету
data_test['target'] = y_test_pred

# Сохраним итоговые данные об id и предсказаниях в формате csv
data_test[['id', 'target']].to_csv('baseline_submission_case1.csv', index=False)